In [ ]:
import pandas as pd
import io
import requests
url = "./data/train_data.csv"
content = requests.get(url).content
df = pd.read_csv(io.StringIO(content.decode('utf-8')))

In [ ]:
df.head()

,prompt,completion
0,Fetch onion,"{'type':'command','expressions':[['|exists x1 ..."
1,Fetch a carrot,"{'type':'command','expressions':[['|exists x1 ..."
2,Fetch 2 avocados,"{'type':'command','expressions':[['|exists x1 ..."
3,Please fetch ten tomatoes,"{'type':'command','expressions':[['|exists x1 ..."
4,Bring a cookie on the kitchen counter,"{'type':'command','expressions':[['|exists x1 ..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   prompt      144 non-null    object
 1   completion  144 non-null    object
dtypes: object(2)
memory usage: 2.4+ KB


In [ ]:
df = df.sample(frac=1)

In [ ]:
df.to_json("training_data.jsonl", orient='records', lines=True)

In [ ]:
df.head()

,prompt,completion
122,There are 2 times more broccoli stems than forks,"{'type':'query','expressions':['|exists x0 (br..."
141,No cheese is feta cheese,"{'type':'query','expressions':['all x0 (cheese..."
57,Line BakingTray1 with baking sheet,"{'type':'command','expressions':[['|exists x1 ..."
41,Slice brownOnion1 with cookingKnife1,"{'type':'command','expressions':[[]],'commands..."
135,There are no onions,"{'type':'query','expressions':['|-(exists x0 (..."


In [ ]:
!pip install --upgrade openai


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 22.7 MB/s eta 0:00:00


In [ ]:
!openai tools fine_tunes.prepare_data -f training_data.jsonl

Analyzing...

- Your file contains 144 prompt-completion pairs
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts empty
- All completions start with prefix `{'type':'`. Most of the time you should only add the output data into the completion, without any prefix
- All completions end with suffix `]}`
- The completion should start with a whitespace character (` `). This tends to produce better results due to the tokenization we use. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more details

Based on the analysis we will perform the following actions:
- [Recommended] Add a suffix separator ` ->` to all prompts 

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-..."

In [ ]:
!openai api fine_tunes.create -t "training_data_prepared.jsonl" -m curie

Upload progress: 100% 27.3k/27.3k [00:00<00:00, 27.9Mit/s]
Uploaded file from training_data_prepared.jsonl: file-mll50XLuoVgz5jxu2IkqU6zX
Created fine-tune: ft-M5TMN9zG9vcXuAgrZSehUYLP
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-05-14 10:35:25] Created fine-tune: ft-M5TMN9zG9vcXuAgrZSehUYLP
[2023-05-14 10:36:03] Fine-tune costs $0.11
[2023-05-14 10:36:03] Fine-tune enqueued. Queue number: 1



In [ ]:
!openai api fine_tunes.follow -i ft-M5TMN9zG9vcXuAgrZSehUYLP

[2023-05-14 10:35:25] Created fine-tune: ft-M5TMN9zG9vcXuAgrZSehUYLP
[2023-05-14 10:36:03] Fine-tune costs $0.11
[2023-05-14 10:36:03] Fine-tune enqueued. Queue number: 1
[2023-05-14 10:37:21] Fine-tune is in the queue. Queue number: 0
[2023-05-14 10:38:45] Fine-tune started
[2023-05-14 10:40:19] Completed epoch 1/4
[2023-05-14 10:40:47] Completed epoch 2/4
[2023-05-14 10:41:15] Completed epoch 3/4
[2023-05-14 10:41:42] Completed epoch 4/4
[2023-05-14 10:41:58] Uploaded model: curie:ft-technical-university-of-cluj-napoca-2023-05-14-10-41-58
[2023-05-14 10:41:59] Uploaded result file: file-M5BmyER6XJjM3wBZWVj5KbGz
[2023-05-14 10:41:59] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m curie:ft-technical-university-of-cluj-napoca-2023-05-14-10-41-58 -p <YOUR_PROMPT>


In [ ]:
import openai
openai.api_key = "sk-..."
ft_model = 'curie:ft-technical-university-of-cluj-napoca-2023-05-14-10-41-58'

In [ ]:
def generate_expression(sentence): 
  res = openai.Completion.create(model=ft_model, prompt=sentence + ' ->', stop=']}', max_tokens=60)
  return res.choices[0].text + ']}'

In [ ]:
test_data = ['Fetch thirteen red onion', 'Fetch BrownOnion1', 'Cut some peppers']

In [ ]:
for sentence in test_data:
  print(generate_expression(sentence))

 {'type':'command','expressions':[['|exists x1 (redOnion(x1)).| >= 13']],'commands':['abe(x0) & onion(x1) -> fetch(x0, x1).']}
 {'type':'command','expressions':[[]],'commands':['abe(x0) -> fetch(x0, BrownOnion1).']}
 {'type':'command','expressions':[['|exists x1 (pepper(x1)).| >= 4']],'commands':['abe(x0) & pepper(x1) & cuttingTool(x2) -> cut(x0, x1]}


In [ ]:
generate_expression('Cut some peppers')

" {'type':'command','expressions':[['|exists x1 (pepper(x1)).| >= 4']],'commands':['abe(x0) & pepper(x1) & cuttingTool(x2) -> cut(x0, x1, x2).']}"

In [ ]:
generate_expression('Cut some avocados using Knife1')

" {'type':'command','expressions':[['|exists x1 (avocado(x1)).| >= 4']],'commands':['abe(x0) & avocado(x1) & cuttingTool( Knife1) -> cut(x0, x1, Knife1).']}"

In [ ]:
generate_expression('Cut Avocado1 using Knife1')

" {'type':'command','expressions':[[]],'commands':['abe(x0) -> cut(x0, Avocado1, Knife1).']}"

In [ ]:
generate_expression('Cut Avocado1 using knife Knife1')

" {'type':'command','expressions':[['|exists x1 (avocado(x1)).| >= 1']],'commands':['abe(x0) & avocado(x1) & cuttingTool(x2) -> cut(x0, x1]}"

In [ ]:
generate_expression('there are more apples than red onions')

" {'type':'query','expressions':['|exists x0 (apple(x0)).| > |exists x0 (redOnion(x0)).|']}"

In [ ]:
generate_expression('there are more apples than red onions and peppers combined')

" {'type':'query','expressions':['|exists x0 (apple(x0)).| > |exists x0 (redOnion(x0)).| + |exists x0 (pepper(x0)).|']}"

In [ ]:
generate_expression('Fetch Onion1')

" {'type':'command','expressions':[[]],'commands':['abe(x0) -> fetch(x0, Onion1).']}"

In [ ]:
generate_expression('Bring 12 apples')

" {'type':'command','expressions':[['|exists x1 (apple(x1)).| >= 12']],'commands':['abe(x0) & apple(x1) -> fetch(x0, x1).']}"